In [1]:
import geopandas as gpd
import pandas as pd

crime_df = pd.read_csv('cleaned_crime.csv')

boston_gdf = gpd.read_file('Boston_Zoning_Subdistricts.csv')
crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(crime_df['Long'], crime_df['Lat']))
print(boston_gdf.shape)
print(boston_gdf.columns)

/var/folders/0f/2vmn8gfj6v973sb5n_jhk8lc0000gn/T/ipykernel_88359/833535223.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('cleaned_crime.csv')


(1649, 15)
Index(['OBJECTID', 'ZONE_', 'DISTRICT', 'MAPNO', 'ARTICLE', 'SUBDISTRIC',
       'Unique_Code', 'FAR', 'Zone_Desc', 'Shape_STArea_1', 'Shape_STLength_1',
       'Urban_Label', 'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object')


## Task 8.1

In [2]:
import folium
import random

crime_gdf['offense_type'] = None
crime_gdf['offense_type'][crime_gdf['NAME'].str.lower(
).str.contains('robbery|assault|rape')] = 'violent crime'
crime_gdf['offense_type'][crime_gdf['NAME'].str.lower(
).str.contains('burglary|larceny|theft|arson')] = 'property crime'

crime_gdf_2019 = crime_gdf[crime_gdf['YEAR']==2019].reset_index()
crime_gdf_2019_property = crime_gdf_2019[crime_gdf_2019['offense_type']=='property crime'].reset_index()
clon, clat = crime_gdf_2019_property['Long'].mean(), crime_gdf_2019_property['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)

#    num indicating the amount of crime 
#    draw_num means drawing the amoiunt of crime in the map

num = len(crime_gdf_2019_property)
draw_num = 50
print("property crime count:", num)
print("draw random:", draw_num)
for i in range(draw_num):
    crime_index = random.randrange(0, num)
    crime_long, crime_lat = crime_gdf_2019_property.loc[crime_index, 'Long'], crime_gdf_2019_property.loc[crime_index, 'Lat']
    folium.CircleMarker((crime_lat, crime_long), opacity=0.7, color='red', weight=4).add_to(m)
display(m)

/var/folders/0f/2vmn8gfj6v973sb5n_jhk8lc0000gn/T/ipykernel_88359/4070141403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_gdf['offense_type'][crime_gdf['NAME'].str.lower(
/var/folders/0f/2vmn8gfj6v973sb5n_jhk8lc0000gn/T/ipykernel_88359/4070141403.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_gdf['offense_type'][crime_gdf['NAME'].str.lower(


property crime count: 24762
draw random: 50


## Task 8.2

In [3]:
crime_gdf_2019_violent = crime_gdf_2019[crime_gdf_2019['offense_type']=='violent crime'].reset_index()
clon, clat = crime_gdf_2019_violent['Long'].mean(), crime_gdf_2019_violent['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)

violent_num = len(crime_gdf_2019_violent)
draw_num = 50
print("property crime count:", violent_num)
print("draw random:", draw_num)
for i in range(draw_num):
    crime_index = random.randrange(0, violent_num)
    crime_long, crime_lat = crime_gdf_2019_violent.loc[crime_index, 'Long'], crime_gdf_2019_violent.loc[crime_index, 'Lat']
    folium.CircleMarker((crime_lat, crime_long), opacity=0.7, color='Blue', weight=4).add_to(m)
display(m)

property crime count: 15780
draw random: 50


## Task 8.3

In [4]:
crime_gdf_property = crime_gdf[crime_gdf['offense_type']=='property crime'].reset_index()
crime_gdf_violent = crime_gdf[crime_gdf['offense_type']=='violent crime'].reset_index()

In [5]:
district_count_property = crime_gdf_property.groupby(['DISTRICT']).agg({'INCIDENT_NUMBER': 'count'}).reset_index()
district_count_violent = crime_gdf_violent.groupby(['DISTRICT']).agg({'INCIDENT_NUMBER': 'count'}).reset_index()

In [6]:
import shapely

def get_centroid(geoserise):
    mp = shapely.geometry.MultiPoint(geoserise)
    return mp.centroid.xy

#getting the id of districts
district_set = set(crime_gdf['DISTRICT'])
district_set.remove("External")

#store the district id into the dictionary 
district_centroid = {}
for d in district_set:
    geoseries = crime_gdf[crime_gdf["DISTRICT"] == d].reset_index()['geometry']
    district_centroid[d] = get_centroid(geoseries)

In [7]:
#removing External
district_count_property.drop(12)
district_count_violent.drop(12)

,DISTRICT,INCIDENT_NUMBER
0,A1,3693
1,A15,466
2,A7,1128
3,B2,4042
4,B3,3245
5,C11,3174
6,C6,1923
7,D14,1309
8,D4,3637
9,E13,1372


In [8]:
district_num = len(district_count_violent)

#getting the maximum and minimum of property and violent crimes in each districts 
p_max, p_min = district_count_property["INCIDENT_NUMBER"].max(), district_count_property["INCIDENT_NUMBER"].min()
v_max, v_min = district_count_violent["INCIDENT_NUMBER"].max(), district_count_violent["INCIDENT_NUMBER"].min()

pv_m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)

district_set = list(district_set)

for i in range(len(district_set)):
    distrcit_name = district_set[i]
    crime_long, crime_lat = district_centroid[distrcit_name][0][0], district_centroid[distrcit_name][1][0]
    
    p_size = district_count_property[district_count_property["DISTRICT"] == distrcit_name]['INCIDENT_NUMBER'].values[0]
    v_size = district_count_violent[district_count_violent["DISTRICT"] == distrcit_name]['INCIDENT_NUMBER'].values[0]
    
    # according to the maximum and minimum, calculating the ratio of crime amounts and using it as the visual indicator on the map
    r_p = ((p_size - p_min) / (p_max - p_min) * 20) + 5
    r_v = ((v_size - v_min) / (v_max - v_min) * 20) + 5
    #property crime as red，violent crimes as green
    folium.CircleMarker((crime_lat, crime_long), opacity=0.7, color='Red', radius=r_p, weight=4).add_to(pv_m)
    folium.CircleMarker((crime_lat, crime_long), opacity=0.7, color='Green', radius=r_v, weight=4).add_to(pv_m)
display(pv_m)